There are various tools available for using the Carto API from python.

** Carto SQL API: **
* https://carto.com/docs/carto-engine/sql-api/

** Review data types in Carto: **
* https://carto.com/docs/tips-and-tricks/data-types/
* Geometry: Points, Multilinestrings, Multipolygons
* Number: double precision type
* String: text field
* Date/Time: modeled after Postgres datetime

Postgres datetime:
* Types: https://www.postgresql.org/docs/9.1/static/datatype-datetime.html
* Functions & Operators: https://www.postgresql.org/docs/9.1/static/functions-datetime.html

Python's datetime, section 8.4: 
* https://docs.python.org/2/library/datetime.html

** Carto Python: **
* https://github.com/CartoDB/carto-python
* Includes support for simplified access to the SQL API and Batch Queries
* Some parts not well documented

** Cartoframes: **
* Github: https://github.com/CartoDB/cartoframes
* Explore the tests to see some advanced usage of carto python library and capabilities of cartoframes: https://github.com/CartoDB/cartoframes/blob/master/test/test_context.py
* Ran into an error using an example from their documentation... bad sign...

** A command line tool that displays OOP design for interacting with Carto tables: **
* https://gist.github.com/andrewxhill/093c89fa45e5f657fec7

Authentication

In [2]:
# No matter what, we will be using authentication credentials:
from configparser import ConfigParser

config = ConfigParser()
config.read("/Users/nathansuberi/Desktop/Code Portfolio/ResourceWatchCode/.env")
# FROM: https://resourcewatch.carto.com/u/wri-rw/your_apps
carto_api_token = config.get("auth", "carto_api_token")

Using the SQL API directly

In [ ]:
# Create table call

# Insert value call

# Truncate table call

# Delete table call

# Import table call

# Add a column call

# Running Batch Jobs

Desired functions for NRT vector processing

In [ ]:
# Reformat data to long form

# Ensure proper time stamp

# Standardize country names, add ISO3

# Add in s2sphere coordinates?

# Upload new rows to Carto table
## Include geometry or not?

# Upload original and edited data to S3



Using the "carto" python package

PACKAGE CONTENTS: auth, datasets, exceptions, export, fields, file_import, maps, paginators, permissions, resources, sql, sync_tables, tables, users, visualizations

In [39]:
import carto
help(carto)

Help on package carto:

NAME
    carto

PACKAGE CONTENTS
    auth
    datasets
    exceptions
    export
    fields
    file_import
    maps
    paginators
    permissions
    resources
    sql
    sync_tables
    tables
    users
    visualizations

FILE
    /Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/carto/__init__.py




In [42]:
# Authenticating
from carto.auth import APIKeyAuthClient

USERNAME="wri-rw"
USR_BASE_URL = "https://{user}.carto.com/".format(user=USERNAME)
auth_client = APIKeyAuthClient(api_key=carto_api_token, base_url=USR_BASE_URL)

In [7]:
# Fetching datasets... 
# Still unclear what this gives besides dataset names
# Warns that this is "non-public" CARTO API and may change in future
from carto.datasets import DatasetManager

dataset_manager = DatasetManager(auth_client)
datasets = dataset_manager.all()

/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/carto/resources.py:90: FutureWarning: This is part of a non-public CARTO API and may change in the future. Take this into account if you are using this in a production environment
  warnings.warn('This is part of a non-public CARTO API and may change in the future. Take this into account if you are using this in a production environment', FutureWarning)


In [47]:
help(datasets[0])
print(datasets[0].__dict__)

Help on Dataset in module carto.datasets object:

class Dataset(carto.resources.WarnResource)
 |  Represents a dataset in CARTO. Typically, that means there is a table in
 |  the PostgreSQL server associated to this object.
 |  
 |  .. warning:: Non-public API. It may change with no previous notice
 |  
 |  Method resolution order:
 |      Dataset
 |      carto.resources.WarnResource
 |      pyrestcli.resources.Resource
 |      pyrestcli.resources.APIConnected
 |      builtins.object
 |  
 |  Data descriptors defined here:
 |  
 |  active_layer_id
 |      Convenient class to make explicit that an attribute will store chars
 |  
 |  attributions
 |      Convenient class to make explicit that an attribute will store chars
 |  
 |  auth_tokens
 |      Convenient class to make explicit that an attribute will store chars
 |  
 |  created_at
 |      Field to store datetimes in resources
 |  
 |  description
 |      Convenient class to make explicit that an attribute will store chars
 |  
 | 

In [46]:
# Use table manager... throws an error
from carto.tables import TableManager
help(TableManager)

table_manager = TableManager(auth_client)
#tables = table_manager.all()

Help on class TableManager in module carto.tables:

class TableManager(carto.resources.Manager)
 |  Manager for the Table class.
 |  
 |  .. warning:: Non-public API. It may change with no previous notice
 |  
 |  Method resolution order:
 |      TableManager
 |      carto.resources.Manager
 |      pyrestcli.resources.Manager
 |      pyrestcli.resources.APIConnected
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  paginator_class = <class 'carto.paginators.CartoPaginator'>
 |      Used internally to retrieve results paginated
 |  
 |  resource_class = <class 'carto.tables.Table'>
 |      Represents a table in CARTO. This is an internal data type. Both Table and
 |      TableManager are not meant to be used outside the SDK
 |      
 |      If you are looking to work with datasets / tables from outside the SDK,
 |      please look into the datasets.py file.
 |      
 |      .. warning:: Non-public API. It may change with no previous notice
 |  
 |  ------

In [ ]:
# Running SQL queries
from carto.sql import SQLClient

sql = SQLClient(auth_client)

try:
    data = sql.send('select * from mytable')
except CartoException as e:
    print("some error ocurred", e)

print data['rows']

In [ ]:
# Running batch queries
from carto.sql import BatchSQLClient

LIST_OF_SQL_QUERIES = []

batchSQLClient = BatchSQLClient(auth_client)
createJob = batchSQLClient.create(LIST_OF_SQL_QUERIES)

print(createJob['job_id'])